#크롤러 모델


In [ ]:
#크롤링 모델 > 아이템 테이블 + 리뷰 테이블 통합 버전

import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# 데려올 카테고리 200000313 / 헤어케어 카테고리
page = [1] # page를 더 추가가 가능함.
p_id = 0 # product_id를 넣을 변수
items = [] # 아이템 데이터를 넣을 리스트
counter_max = 40

for pg in page: # 원하는 페이지 수만큼 반복
    resp = requests.get(f'https://browse.gmarket.co.kr/list?category=200000313&s=13&k=0&p={pg}') # s = 13 -> 리뷰 많은 순으로 정렬
    soup = BeautifulSoup(resp.content, 'html.parser')

    item_name = soup.select('div#content div.box__item-title span.text__item')
    item_url = soup.select("div#content span.text__item-title a.link__item")
    item_price = soup.select("div#content div.box__price-seller strong.text")
    counter = 0
    # 아이템 테이블
    for name,urls,prc in zip(item_name,item_url,item_price):
        # 각 아이템별 내용을 저장할 딕셔너리 data
        if counter < counter_max:
            p_id += 1
            data = {
                    "product_id" : p_id, # 아이템 id를 지정
                    "product_name": name['title'], # 아이템 이름
                    "price": prc.text, # 가격
                    "url_link" : urls['href'], # 아이템 별 세부페이지 url 소스
            }
            items.append(data)
        counter += 1

# 데이터 프레임 만들기
item_frame = pd.DataFrame(items)

###############################################

headers = {
        'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '42',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    }

review_dic_list = []
img_list = []
detail_img_list = []


for url,name, p_id in tqdm(zip(item_frame['url_link'], item_frame['product_name'], item_frame['product_id'])):
    pg_num = 0

    #프리미엄 상품평 최대 페이지 수 구하기
    resp1 = requests.post('http://item.gmarket.co.kr/Review',
                        data='goodsCode={}&pageNo=1&totalPage=1000'.format(url[41:]),
                        headers=headers)
    soup_page = BeautifulSoup(resp1.text)
    page_number = soup_page.select('div#premium-wrapper div.board_pagenationwrap div.board_paging span.pagetotal em')
    for pg in page_number:
        pg_num = int(pg.text) # 최대 페이지 수 pg_num

    ####################################

    # 상품 썸네일 이미지 크롤링
    resp_img = requests.get(f'http://item.gmarket.co.kr/Item?goodscode={url[41:51]}')
    soup_img = BeautifulSoup(resp_img.text)
    thumb_img = soup_img.select('div.thumb-gallery img')
    for img in thumb_img:
        img_list.append(img['src']) # 이미지 url 한개만 크롤링
        break
    ####################################

    # 상세설명 이미지 크롤링
    resp2 = requests.post('http://item.gmarket.co.kr/Item/ItemDetail',
                            data='goodsCode={}'.format(url[41:]),
                        headers=headers)
    soup_detail_img = BeautifulSoup(resp2.content, 'html.parser')
    i_img = soup_detail_img.select('div#basic_detail_html img')
    detail_item_count = 0
    for i in i_img:
        detail_item_count += 1
        # 몇번째 사진을 가져올 것 인가?
        if detail_item_count == 1: # 1번째 사진
            detail_img = i['src']
            break
    detail_img_dic = {"detail_img" : detail_img}
    detail_img_list.append(detail_img_dic)
    ####################################

    # 프리미엄 리뷰 크롤링
    for pgs in range(1, pg_num): # 1~20 페이지까지 크롤링(페이지 당 5개의 리뷰)
        payload = {
            'goodsCode': int(url[41:]), # 아이템 넘버
            'pageNo': pgs, # 리뷰 페이지
            'sort': 0, # 정렬방식
            'totalPage': pg_num # 리뷰 페이지의 끝
        } #payload를 통해 pgs와 url링크를 변환해서 접근

        # Review/Premium에서 크롤링
        resp3 = requests.post('http://item.gmarket.co.kr/Review/Premium',
                            payload,
                            headers=headers)
        soup_review = BeautifulSoup(resp3.text)

        reviews = soup_review.select('p.con') # 리뷰 텍스트
        reviewer_ids = soup_review.select('dl.writer-info dd') #리뷰어 id
        reviewer_id = reviewer_ids[::3] # reviewer_ids 를 크롤링 시에 날짜와 조회수가 같이 크롤링되어서 [::3]으로 id값만 추출

        for review_id, review_text in zip(reviewer_id, reviews): # 페이지 당 5개의 리뷰
            review_data = {
                    "PRODUCT_ID" : p_id, #index
                    "PRODUCT_NAME": name, # 아이템 명
                    "ID" : review_id.text, #리뷰 유저 id
                    "REVIEW": review_text.text #리뷰 텍스트
            }
            review_dic_list.append(review_data)

#csv변환을 위한 데이터프레임화
detail_img_df = pd.DataFrame(detail_img_list)
thumb_img_df = pd.DataFrame(img_list)

# 아이템 테이블에 이미지 데이터프레임 합치기
item_frame_with_img = pd.concat([item_frame, thumb_img_df], axis = 1)
item_frame_result = pd.concat([item_frame_with_img, detail_img_df], axis = 1)

#이름 맞춰주기
item_frame_result.rename(columns = {'product_id' : 'PRODUCT_ID', 'product_name': 'PRODUCT_NAME','price':'PRICE',
                                    'url_link':'URL_LINK',0:'ITEM_IMG', 'detail_img': 'DETAIL_IMG'}, inplace = True)
#csv로 파일만들어주기
item_frame_result.to_csv("item_gmarket.csv")

review_dic_list = pd.DataFrame(review_dic_list)
review_dic_list.to_csv("review_gmarket.csv")

#두 가지 카테고리 크롤링

In [ ]:
#크롤링 모델 > 2가지 카테고리 크롤링을 한 테이블에

import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# 데려올 카테고리 200001481 / 헤어스타일링 카테고리
category = [200000313,200001481]
page = [1] # page를 더 추가가 가능함.
p_id = 0 # product_id를 넣을 변수

counter_max = 20

review_dic_list = []
img_list = []
detail_img_list = []
items = [] # 아이템 데이터를 넣을 리스트

for cat in category:
    for pg in page: # 원하는 페이지 수만큼 반복
        resp = requests.get(f'https://browse.gmarket.co.kr/list?category={cat}&s=13&k=0&p={pg}') # s = 13 -> 리뷰 많은 순으로 정렬
        soup = BeautifulSoup(resp.content, 'html.parser')

        item_name = soup.select('div#content div.box__item-title span.text__item')
        item_url = soup.select("div#content span.text__item-title a.link__item")
        item_price = soup.select("div#content div.box__price-seller strong.text")
        counter = 0
        # 아이템 테이블
        for name,urls,prc in zip(item_name,item_url,item_price):
            # 각 아이템별 내용을 저장할 딕셔너리 data
            if counter < counter_max:
                p_id += 1
                data = {
                        "CATEGORY_ID" : cat,
                        "product_id" : p_id, # 아이템 id를 지정
                        "product_name": name['title'], # 아이템 이름
                        "price": prc.text, # 가격
                        "url_link" : urls['href'], # 아이템 별 세부페이지 url 소스
                }
                items.append(data)
            counter += 1

# 데이터 프레임 만들기
item_frame = pd.DataFrame(items)

###############################################

headers = {
        'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '42',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    }
review_dic_list = []
img_list = []
detail_img_list = []

for cat,url,name, p_id in tqdm(zip(item_frame['CATEGORY_ID'],item_frame['url_link'], item_frame['product_name'], item_frame['product_id'])):
    pg_num = 0

    #프리미엄 상품평 최대 페이지 수 구하기
    resp1 = requests.post('http://item.gmarket.co.kr/Review',
                        data='goodsCode={}&pageNo=1&totalPage=1000'.format(url[41:]),
                        headers=headers)
    soup_page = BeautifulSoup(resp1.text)
    page_number = soup_page.select('div#premium-wrapper div.board_pagenationwrap div.board_paging span.pagetotal em')
    for pg in page_number:
        pg_num = int(pg.text) # 최대 페이지 수 pg_num

    ####################################

    # 상품 썸네일 이미지 크롤링
    resp_img = requests.get(f'http://item.gmarket.co.kr/Item?goodscode={url[41:51]}')
    soup_img = BeautifulSoup(resp_img.text)
    thumb_img = soup_img.select('div.thumb-gallery img')
    for img in thumb_img:
        img_list.append(img['src']) # 이미지 url 한개만 크롤링
        break
    ####################################

    # 상세설명 이미지 크롤링
    resp2 = requests.post('http://item.gmarket.co.kr/Item/ItemDetail',
                            data='goodsCode={}'.format(url[41:]),
                        headers=headers)
    soup_detail_img = BeautifulSoup(resp2.content, 'html.parser')
    i_img = soup_detail_img.select('div#basic_detail_html img')
    detail_item_count = 0
    for i in i_img:
        detail_item_count += 1
        # 몇번째 사진을 가져올 것 인가?
        if detail_item_count == 1: # 1번째 사진
            detail_img = i['src']
            break
    detail_img_dic = {"detail_img" : detail_img}
    detail_img_list.append(detail_img_dic)
    ####################################

    # 프리미엄 리뷰 크롤링
    for pgs in range(1, pg_num): # 1~20 페이지까지 크롤링(페이지 당 5개의 리뷰)
        payload = {
            'goodsCode': int(url[41:]), # 아이템 넘버
            'pageNo': pgs, # 리뷰 페이지
            'sort': 0, # 정렬방식
            'totalPage': pg_num # 리뷰 페이지의 끝
        } #payload를 통해 pgs와 url링크를 변환해서 접근

        # Review/Premium에서 크롤링
        resp3 = requests.post('http://item.gmarket.co.kr/Review/Premium',
                            payload,
                            headers=headers)
        soup_review = BeautifulSoup(resp3.text)

        reviews = soup_review.select('p.con') # 리뷰 텍스트
        reviewer_ids = soup_review.select('dl.writer-info dd') #리뷰어 id
        reviewer_id = reviewer_ids[::3] # reviewer_ids 를 크롤링 시에 날짜와 조회수가 같이 크롤링되어서 [::3]으로 id값만 추출

        for review_id, review_text in zip(reviewer_id, reviews): # 페이지 당 5개의 리뷰
            review_data = {
                    "CATEGORY_ID" : cat,
                    "PRODUCT_ID" : p_id, #index
                    "PRODUCT_NAME": name, # 아이템 명
                    "ID" : review_id.text, #리뷰 유저 id
                    "REVIEW": review_text.text #리뷰 텍스트
            }
            review_dic_list.append(review_data)

#csv변환을 위한 데이터프레임화
detail_img_df = pd.DataFrame(detail_img_list)
thumb_img_df = pd.DataFrame(img_list)

# 아이템 테이블에 이미지 데이터프레임 합치기
item_frame_with_img = pd.concat([item_frame, thumb_img_df], axis = 1)
item_frame_result = pd.concat([item_frame_with_img, detail_img_df], axis = 1)

#이름 맞춰주기
item_frame_result.rename(columns = {'product_id' : 'PRODUCT_ID', 'product_name': 'PRODUCT_NAME','price':'PRICE',
                                    'url_link':'URL_LINK',0:'ITEM_IMG', 'detail_img': 'DETAIL_IMG'}, inplace = True)
#csv로 파일만들어주기
item_frame_result.to_csv("item_gmarket_hairstyle.csv")

review_dic_list = pd.DataFrame(review_dic_list)
review_dic_list.to_csv("review_gmarket_hairstyle.csv")

40it [1:21:23, 122.08s/it]


#크롤링 모델(아이템 테이블만)


In [ ]:
#크롤링 모델 > 아이템 테이블

import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# 데려올 카테고리 200000313 => 헤어케어
page = [1]
p_id = 0
items = []

for pg in page:
    resp = requests.get(f'https://browse.gmarket.co.kr/list?category=200000313&s=13&k=0&p={pg}') #s = 13 --> 리뷰 많은 순.
    # 크롤링
    soup = BeautifulSoup(resp.content, 'html.parser')

    item_name = soup.select('div#content div.box__item-title span.text__item')
    item_url = soup.select("div#content span.text__item-title a.link__item") # url
    item_price = soup.select("div#content div.box__price-seller strong.text")

    # 아이템 설명 테이블
    for name,urls,prc in zip(item_name,item_url,item_price):
        # 각 아이템별 내용을 저장할 딕셔너리 data
        p_id += 1
        data = {
                "product_id" : p_id, #index
                "product_name": name['title'], # 이름
                "price": prc.text, # 가격
                "url_link" : urls['href'], # 아이템 별 세부페이지 url 소스
        }
        items.append(data)

# 데이터 프레임 만들기
item_frame = pd.DataFrame(items)

headers = {
        'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '42',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    }

img_list = []
detail_img_list = []

for url,name, p_id in tqdm(zip(item_frame['url_link'], item_frame['product_name'], item_frame['product_id'])): #리뷰 데이터 모으려고 만든 것이라서 불필요한 매개변수 존재
    pg_num = 0
    review_list = []
    review_id_list = []

    resp_img = requests.get(f'http://item.gmarket.co.kr/Item?goodscode={url[41:51]}')
    soup_img = BeautifulSoup(resp_img.text)
    thumb_img = soup_img.select('div.thumb-gallery img')

    for img in thumb_img:
        img_list.append(img['src'])
        break

    # 상세설명 이미지 크롤링
    resp2 = requests.post('http://item.gmarket.co.kr/Item/ItemDetail',
                            data='goodsCode={}'.format(url[41:51]),
                        headers=headers)
    soup_detail_img = BeautifulSoup(resp2.content, 'html.parser')
    i_img = soup_detail_img.select('div#basic_detail_html img')
    detail_item_count = 0
    for i in i_img:
        detail_item_count += 1
        # 몇번째 사진을 가져올 것 인가?
        if detail_item_count == 1: # 1번째 사진
            detail_img = i['src']
            break
    detail_img_dic = {"detail_img" : detail_img}
    detail_img_list.append(detail_img_dic)

detail_img_df = pd.DataFrame(detail_img_list)
thumb_img_df = pd.DataFrame(img_list)

item_frame_with_img = pd.concat([item_frame, thumb_img_df], axis = 1)
item_frame_result = pd.concat([item_frame_with_img, detail_img_df], axis = 1)

item_frame_result.rename(columns = {'product_id' : 'PRODUCT_ID', 'product_name': 'PRODUCT_NAME','price':'PRICE',
                                    'url_link':'URL_LINK',0:'ITEM_IMG', 'detail_img': 'DETAIL_IMG'}, inplace = True)
item_frame_result.to_csv("item_gmarket.csv")

100it [05:34,  3.35s/it]
